# Fine-Tune Whisper

Adapted from guide here: https://huggingface.co/blog/fine-tune-whisper

## Load Data

In [ ]:
from datasets import load_from_disk


ds = load_from_disk('processed-amdo-ds')

amdo_test = ds['test']
kham_test = load_from_disk('Data/processed-kham-test-ds')
utsang_test = load_from_disk('Data/processed-utsang-test-ds')

## Load Processors and Model

In [ ]:
from transformers import WhisperProcessor, WhisperFeatureExtractor, WhisperTokenizer, WhisperForConditionalGeneration

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-tiny", language="bo", task="transcribe")
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-tiny")
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny", language="bo", task="transcribe")

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")

In [ ]:
model.generation_config.language = "bo"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

### Define a Data Collator

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

Let's initialise the data collator we've just defined:

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

### Evaluation Metrics

In [ ]:
import evaluate
from tibetan_wer.wer import wer as tib_wer

cer_metric = evaluate.load("cer")

In [ ]:
import pickle

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    tib_wer_res = tib_wer(predictions=pred_str, references=label_str)

    macro_wer = tib_wer_res['macro_wer']
    micro_wer = tib_wer_res['micro_wer']
    subs = tib_wer_res['substitutions']
    ins = tib_wer_res['insertions']
    dels = tib_wer_res['deletions']


    return {"cer": cer,
            "tib_macro_wer": macro_wer,
            "tib_micro_wer": micro_wer,
            "substitutions": subs,
            "insertions": ins,
            "deletions": dels
            }

### Define the Training Configuration and Train

In [ ]:
%env WANDB_PROJECT=asr-dialect-lora

learning rate from: https://github.com/vasistalodagala/whisper-finetune

In [ ]:
ds['train'].select(range(10))

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="baseline-amdo",
    auto_find_batch_size=True,
    #per_device_train_batch_size=4,
    gradient_accumulation_steps=4,  # increase by 2x for every 2x decrease in batch size
    learning_rate=3.75e-5, # tiny: 3.75e-5, base: 2.5e-5, tiny: 1.25e-5, med: 6.25e-6, large: 4.375e-6
    gradient_checkpointing=True,
    fp16=True,
    #per_device_eval_batch_size=2,
    predict_with_generate=True,
    generation_max_length=225,
    evaluation_strategy='epoch',
    #load_best_model_at_end=True,
    metric_for_best_model="cer",
    greater_is_better=False,
    num_train_epochs=3
)

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=ds['train'],
    eval_dataset={'Amdo': amdo_test, 'Kham': kham_test, 'Utsang': utsang_test},
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

trainer.train()

In [ ]:
#model.save_pretrained('baseline-amdo-finetuned')